In [16]:
def match_keys(data, valid, path):
    data_keys = data.keys()
    valid_keys = valid.keys()
    
    extra_keys = data_keys - valid_keys
    missing_keys = valid_keys - data_keys
    
    if missing_keys or extra_keys:
        missing_msg = ('missing keys: '+ 
                      ', '.join({path + '.' + str(key)
                               for key in missing_keys})
                      ) if missing_keys else ''
        extras_msg = ('extra keys: ' +
                     ', '.join({path + '.' + str(key)
                              for key in extra_keys})
                     ) if extra_keys else ''
        
        return False, ' '.join((missing_msg, extras_msg))
    else:
        return True, None

In [32]:
john = {
    'user_id': 100,
    'name': {
        'first': 'John',
        'last': 'Cleese'
    },
    'bio': {
        'dob': {
            'year': 1939,
            'month': 11,
            'day': 27
        },
        'birthplace': {
            'country': 'United Kingdom',
            'city': 'Weston-super-Mare'
        }
    }
}

In [33]:
temp = {
    'user_id': int,
    'name': {
        'first': str,
        'last': str
    },
    'bio': {
        'dob': {
            'year': int,
            'month': int,
            'day': int
        },
        'birthplace': {
            'country': str,
            'city': str
        }
    }
}

In [35]:
eric = {
    'user_id': 101,
    'name': {
        'first': 'Eric',
        'last': 'Idle'
    },
    'bio': {
        'dob': {
            'year': 1943,
            'month': 3,
            'day': 29
        },
        'birthplace': {
            'country': 'United Kingdom'
        }
    }
}

In [38]:
michael = {
    'user_id': 102,
    'name': {
        'first': 'Michael',
        'last': 'Palin'
    },
    'bio': {
        'dob': {
            'year': 1943,
            'month': 'May',
            'day': 5
        },
        'birthplace': {
            'country': 'United Kingdom',
            'city': 'Sheffield'
        }
    }
}

In [17]:
t = {'a': int, 'b': int, 'c': int, 'd': {}}
d = {'a': 'wrong type', 'b': 100, 'c': 200, 'd': {'wrong', 'type'}}
is_ok, err_msg = match_keys(d,t,'some.path')
print(is_ok, err_msg)

True None


In [18]:
d = {'a': None, 'b': None, 'c': None}
is_ok, err_msg = match_keys(d,t,'some.path')
print(is_ok, err_msg)

False missing keys: some.path.d 


In [19]:
d = {'a': None, 'b': None, 'c': None,  'e': None}
is_ok, err_msg = match_keys(d,t,'some.path')
print(is_ok, err_msg)

False missing keys: some.path.d extra keys: some.path.e


In [20]:
d = {'a':None, 'b':None, 'e': None, 'f': None}
is_ok, err_msg = match_keys(d,t,'some.path')
print(is_ok, err_msg)

False missing keys: some.path.d, some.path.c extra keys: some.path.e, some.path.f


In [27]:
def match_types(data, temp, path):
    for k, v in temp.items():
        if isinstance(v, dict):
            temp_type = dict
        else:
            temp_type = v
        data_val = data.get(k,object())
        
        if not isinstance(data_val, temp_type):
            error_msg = ('incorrect type: ' +path+ '.'+ k
                        + '--> expected ' + temp_type.__name__+
                        ', found ' + type(data_val).__name__)
            return False, error_msg
    return True, None

In [28]:
t = {'a': int, 'b': str, 'c': {'d': int}}
d = {'a':100, 'b': 'test', 'c': {'some': 'val'}}
match_types(d,t,'some.path')

(True, None)

In [29]:
d = {'a':100, 'b': 'test', 'c': 'unexpected'}
match_types(d,t,'some.path')

(False, 'incorrect type: some.path.c--> expected dict, found str')

In [30]:
d = {'a':100, 'b': 200, 'c': {}}
match_types(d,t,'some.path')

(False, 'incorrect type: some.path.b--> expected str, found int')

In [31]:
def recurse_val(data, temp, path):
    is_ok, error_msg = match_keys(data, temp, path)
    if not is_ok:
        return False, error_msg
    
    is_ok, error_msg = match_types(data, temp,path)
    if not is_ok:
        return False, error_msg
    
    diction_type_keys = {key for key, val in temp.items()
                        if isinstance(val,dict)}
    
    for key in diction_type_keys:
        sub_path = path + '.' + str(key)
        sub_temp = temp[key]
        sub_data = data[key]
        is_ok, error_msg = recurse_val(sub_data, sub_temp, sub_path)
        if not is_ok:
            return False, error_msg
        
        
    return True, None

In [34]:
is_ok, error_msg = recurse_val(john, temp, 'root')
print(is_ok, error_msg)

True None


In [36]:
is_ok, error_msg = recurse_val(eric, temp, 'root')
print(is_ok, error_msg)

False missing keys: root.bio.birthplace.city 


In [39]:
is_ok, error_msg = recurse_val(michael, temp, 'root')
print(is_ok, error_msg)

False incorrect type: root.bio.dob.month--> expected int, found str


In [40]:
def val(data, temp):
    return recurse_val(data, temp, '')

In [41]:
persons = (john, 'John'), (eric, 'Eric'), (michael, 'Michael')

In [44]:
for person, name in persons:
    is_ok, error_msg = val(person, temp)
    print(f'{name}: valid={is_ok}: {error_msg}')

John: valid=True: None
Eric: valid=False: missing keys: .bio.birthplace.city 
Michael: valid=False: incorrect type: .bio.dob.month--> expected int, found str


In [45]:
class SchemaError(Exception):
    pass

In [46]:
def val(data, temp):
    is_ok, error_msg = recurse_val(data, temp, '')
    if not is_ok:
        raise SchemaError(error_msg)

In [48]:
val(john, temp)

In [50]:
try:
    for person, name in persons:
        val(person, temp)
except SchemaError as ex:
    print('Validation failed',  str(ex))

Validation failed missing keys: .bio.birthplace.city 


In [53]:
class SchemaKeyMismatch(SchemaError):
    pass

class SchemaTypeMismatch(SchemaError, TypeError):
    pass

In [58]:
def match_keys(data, valid, path):
    data_keys = data.keys()
    valid_keys = valid.keys()
    
    extra_keys = data_keys - valid_keys
    missing_keys = valid_keys - data_keys
    
    if missing_keys or extra_keys:
        missing_msg = ('missing keys: '+ 
                      ', '.join({path + '.' + str(key)
                               for key in missing_keys})
                      ) if missing_keys else ''
        extras_msg = ('extra keys: ' +
                     ', '.join({path + '.' + str(key)
                              for key in extra_keys})
                     ) if extra_keys else ''
        raise SchemaKeyMismatch(' '.join((missing_msg, extras_msg)))

In [59]:
def match_types(data, temp, path):
    for k, v in temp.items():
        if isinstance(v, dict):
            temp_type = dict
        else:
            temp_type = v
        data_val = data.get(k,object())
        
        if not isinstance(data_val, temp_type):
            error_msg = ('incorrect type: ' +path+ '.'+ k
                        + '--> expected ' + temp_type.__name__+
                        ', found ' + type(data_val).__name__)
            raise SchemaTypeMismatch(error_msg)

In [60]:
def recurse_val(data, temp, path):
    match_keys(data, temp, path)
    match_types(data, temp,path)
    diction_type_keys = {key for key, val in temp.items()
                        if isinstance(val,dict)}
    
    for key in diction_type_keys:
        sub_path = path + '.' + str(key)
        sub_temp = temp[key]
        sub_data = data[key]
        recurse_val(sub_data, sub_temp, sub_path)

In [61]:
def val(data, temp):
    recurse_val(data, temp, '')

In [63]:
val(john,temp)

In [65]:
val(eric,temp)

SchemaKeyMismatch: missing keys: .bio.birthplace.city 

In [66]:
val(michael,temp)

SchemaTypeMismatch: incorrect type: .bio.dob.month--> expected int, found str

In [67]:
try:
    val(john, temp)
except SchemaError as ex:
    print(ex)

In [68]:
try:
    val(eric, temp)
except SchemaError as ex:
    print(ex)

missing keys: .bio.birthplace.city 


In [69]:
try:
    val(michael, temp)
except SchemaError as ex:
    print(ex)

incorrect type: .bio.dob.month--> expected int, found str


In [72]:
try:
    val(michael,temp)
except SchemaTypeMismatch as ex:
    print(ex)

incorrect type: .bio.dob.month--> expected int, found str


In [73]:
try:
    val(michael, temp)
except SchemaKeyMismatch as ex:
    print('handling a key mismatch exception')
except SchemaTypeMismatch as ex:
    print('handling a type mismatch exception', ex)
except SchemaError as ex:
    print('handling some general schema exception', ex)
except TypeError as ex:
    print('handling a general type exception', ex)

handling a type mismatch exception incorrect type: .bio.dob.month--> expected int, found str
